In [2]:
# code for autoreload of modules
%load_ext autoreload
%autoreload 2

In [3]:
import os
# Set your custom working DIR:
DIR = '/Users/jannispoltier/Documents/Studium/3. Semester/tabpfn_credit_codebase'
# DIR = r'C:\Users\u0148775\PycharmProjects\CreditScoring'
os.chdir(DIR)



In [4]:
from src.utils import set_random_seed
set_random_seed(0)

In [5]:
from src.utils import load_config
from pathlib import Path

dataconfig = load_config(Path('config') / 'CONFIG_DATA.yaml')
experimentconfig = load_config(Path('config') / 'CONFIG_EXPERIMENT.yaml')
methodconfig = load_config(Path('config') / 'CONFIG_METHOD.yaml')
evaluationconfig = load_config(Path('config') / 'CONFIG_EVALUATION.yaml')
tuningconfig = load_config(Path('config') / 'CONFIG_TUNING.yaml')
paramconfig = load_config(Path('config') / 'datasets' / 'OPTIMAL_HYPERPARAS.yaml')


In [8]:
from src.classes.experiment import Experiment

experiment = Experiment(dataconfig=dataconfig,
                        experimentconfig=experimentconfig,
                        methodconfig=methodconfig,
                        evaluationconfig=evaluationconfig,
                        tuningconfig=tuningconfig,
                        paramconfig=paramconfig)

2025-06-15 20:25:31,424 | INFO | Experiment created at: 2025-06-15-20-25
2025-06-15 20:25:31,425 | INFO | Task: pd
2025-06-15 20:25:31,425 | INFO | Dataset: 01_gmsc
2025-06-15 20:25:31,425 | INFO | CV splits: 3


In [10]:
experiment.run()

2025-06-15 20:25:44,543 | INFO | 01_gmsc loaded
2025-06-15 20:25:44,544 | INFO | Dataset has 150000 rows. Subsampling to 30000 rows.
2025-06-15 20:25:44,548 | INFO | Inducing class imbalance with ratio 0.1 ...
2025-06-15 20:25:44,548 | INFO | Class distribution BEFORE imbalance: {0: 935, 1: 65}
2025-06-15 20:25:44,549 | INFO | Majority class: 0, Minority class: 1
2025-06-15 20:25:44,549 | INFO | Will keep 65 of 65 minority samples (ratio 0.1)
2025-06-15 20:25:44,550 | INFO | Class distribution AFTER imbalance: {0: 935, 1: 65}
2025-06-15 20:25:44,550 | INFO | Total samples: 1000



01_gmsc preprocessed
x shape:  (1000, 10)
y shape:  (1000,)


Cross-validation loop::   0%|          | 0/3 [00:00<?, ?it/s]


NameError: name 'logger' is not defined

In [ ]:
experiment.results

In [ ]:

# Todo: implement method that writes experiment.results to a csv file / connects with WandB / ...

# Quick and dirty examplary code:

import pandas as pd

# Convert dictionary of dictionaries to a list of rows
rows = []
for model_name, splits_dict in experiment.results.items():
    for split_idx, metrics_dict in splits_dict.items():
        # Create a row dict with model and split identifiers
        row = {
            'model': model_name,
            'split': split_idx,
        }
        # Update with all metrics in that split
        row.update(metrics_dict)

        rows.append(row)

# Convert the list of row dictionaries into a DataFrame
df = pd.DataFrame(rows)

df

# Summarise df by model (along split) and get the mean and sd of all the metrics, discarding the split and sorting by mean of aucroc
df_summary = df.groupby('model').agg(['mean', 'std']).reset_index()
# df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('split')]
df_summary


